In [ ]:
!pip install langchain langchain_community language-tool-python


In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
import csv


# 모델 생성

In [ ]:
# llm = ChatOllama(model="Lama3.2-korean:latest", max_token = 500)
llm = ChatOllama(model="llama3.2", max_token = 500)

# 프롬프트 

In [ ]:
# 포함할 단어 목록
words = ["apple", "school", "book", "play", "friend", "happy", "jump", "sun", "water", "cat"]

# PromptTemplate 설정
prompt_template = PromptTemplate(
    input_variables=["word"],
    template="Write a simple English sentence that an elementary school student can understand. The sentence should include the word '{word}' and nothing else."
)   


# 출력 파서

In [ ]:
output_parser = CommaSeparatedListOutputParser()

# 체인 생성

In [ ]:
chain = prompt_template | llm | output_parser


# 실행 코드(테스트)

In [ ]:
# CSV 파일 생성
from random import randint

import language_tool_python

# LanguageTool 인스턴스 생성 (영어로 설정)
tool = language_tool_python.LanguageTool('en-US')

with open('grammar_sentences.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["문장 번호", "문장", "포함된 단어"])

    # 각 단어에 대해 문장 생성 및 저장
    for idx, word in enumerate(words, start=1):
        rand = randint(1,6)
        response = chain.invoke({"word": word})  # 체인 실행
        
        # 파싱된 문장 기록
        for sentence in response:
            writer.writerow([idx, sentence, word])
            print(response)
            
            # 오류 검사
            matches = tool.check(sentence)

            # 오류 출력
            if matches:
                print(f"발견된 오류 수: {len(matches)}")
                for match in matches:
                    print(f"오류: {match.message}")
                    print(f"제안된 수정: {match.replacements}")
                    print(f"오류 위치: {sentence[match.offset : match.offset + match.errorLength]}\n")
            else:
                print("문장에 오류가 없습니다.")

            # 검사기가 제안한 수정 적용
            corrected_sentence = language_tool_python.utils.correct(sentence, matches)
            print("수정된 문장:", corrected_sentence)

In [ ]:
import language_tool_python

# LanguageTool 인스턴스 생성 (영어로 설정)
tool = language_tool_python.LanguageTool('en-US')

# 검증할 문장
sentence = "This is an example sentence with a mistkae."


